# Oļegs Korsaks, 051RDB146

In [1]:
from math import exp
from operator import methodcaller

import numpy as np
import tensorflow as tf

from sklearn.datasets import load_iris

In [2]:
letters = [
    ['A',[0,1,0,1,0,1,1,1,1,1,0,1,1,0,1],[1,0,0]],
    ['E',[1,1,1,1,0,0,1,1,0,1,0,0,1,1,1],[1,0,0]],
    ['D',[1,1,0,1,0,1,1,0,1,1,0,1,1,1,0],[1,0,0]],
    ['B',[1,1,0,1,0,1,1,1,0,1,0,1,1,1,0],[0,1,0]],
    ['C',[1,1,1,1,0,0,1,0,0,1,0,0,1,1,1],[0,1,0]],
    ['F',[1,1,1,1,0,0,1,1,0,1,0,0,1,0,0],[0,1,0]],
    ['G',[1,1,1,1,0,0,1,0,1,1,0,1,1,1,1],[0,0,1]],
    ['H',[1,0,1,1,0,1,1,1,1,1,0,1,1,0,1],[0,0,1]],
    ['I',[0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],[0,0,1]],
]

In [3]:
def a_step(val):
    return int(val>0)


def a_lin(val):
    if val <= 0:
        return 0
    elif val > 5:
        return 1
    
    return val/5


def a_sigmoid(val):
    return 1.0 / (1.0+exp(-val))


def a_tanh(val):
    return (exp(val)-exp(-val)) / (exp(val)+exp(-val))

In [4]:
learning_rate = 0.1


class Neiron:
    def __init__(self, in_size):
        self.weights = np.zeros(in_size)
        self.w0 = 0

    def classify(self, incoming_signals):
        net = np.dot(incoming_signals, self.weights) + self.w0
        
        return a_step(net)
    
    def learn(self, incoming_signals, c):
        y = self.classify(incoming_signals)
        delta = c - y
        
        if delta:
            for i in range(len(self.weights)):
                self.weights[i] += delta * learning_rate * incoming_signals[i]

            self.w0 += delta * learning_rate

In [5]:
def run_neirons(data: list, neiron_class, in_size: int, class_count: int, epoch_count: int = 100):
    neirons = tuple(neiron_class(in_size) for _ in range(class_count))
    
    for epoch in range(epoch_count):
        for sample in data:
            for i, neiron in enumerate(neirons):
                neiron.learn(sample[1], sample[2][i])
    
    for sample in data:
        method = methodcaller('classify', sample[1])
        print(f'{sample[0]} {tuple(map(method, neirons))}')
    
    for neiron in neirons:
        print(neiron.weights)

In [6]:
run_neirons(letters, Neiron, 15, 3, 100)

A (1, 0, 0)
E (1, 0, 0)
D (1, 0, 0)
B (0, 1, 0)
C (0, 1, 0)
F (0, 1, 0)
G (0, 0, 1)
H (0, 0, 1)
I (0, 0, 1)
[-0.7  0.5 -0.8  0.  -0.6 -0.3  0.   0.9  1.8  0.  -0.6 -1.   0.   0.3
  0.4]
[ 0.2  0.2 -0.4  0.2 -0.2  0.4  0.2 -0.7 -1.7  0.2 -0.2 -0.1  0.2  0.2
 -0.4]
[ 0.2 -0.3  0.3 -0.2  0.3 -0.1 -0.2 -0.2  0.4 -0.2  0.3  0.4 -0.2  0.1
 -0.1]


In [7]:
iris = load_iris()
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(4, )))
model.add(tf.keras.layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy')

targets = np.zeros((len(iris.target), 3))

for ti, iris_target in enumerate(iris.target):
    targets[ti][iris_target] = 1

model.fit(iris.data, targets, epochs=1000, verbose=0)
predictions = model.predict(iris.data)

print(predictions)

[[0.93464    0.0556077  0.00975225]
 [0.93001294 0.0592729  0.01071412]
 [0.9418881  0.04980503 0.00830679]
 [0.92969114 0.05952506 0.01078376]
 [0.93753755 0.05329685 0.00916556]
 [0.90686554 0.07731038 0.01582398]
 [0.93770045 0.05316234 0.00913722]
 [0.92806494 0.06081424 0.01112083]
 [0.9361078  0.05442955 0.0094626 ]
 [0.92787623 0.06096046 0.01116338]
 [0.92639464 0.06213566 0.0114698 ]
 [0.9243358  0.06374516 0.01191905]
 [0.9344731  0.05573552 0.00979148]
 [0.95570326 0.03856879 0.00572802]
 [0.94340414 0.04859576 0.00800015]
 [0.9267679  0.06184871 0.01138334]
 [0.93671405 0.05395901 0.00932697]
 [0.93177116 0.05788578 0.01034303]
 [0.9021639  0.08091307 0.01692293]
 [0.9298747  0.05938864 0.01073666]
 [0.90415525 0.07938366 0.01646101]
 [0.9250767  0.0631681  0.01175512]
 [0.96047133 0.03463266 0.00489602]
 [0.89559525 0.08589385 0.01851095]
 [0.89861953 0.08359656 0.01778397]
 [0.91306835 0.07251967 0.01441198]
 [0.9135327  0.07216311 0.01430419]
 [0.92639554 0.06213161 0.01

## Mājasdarbs 1

Pārveidosim neirona klasi, lai viņš varētu klasificēt 4 klašu simbolus. Pievienosim ! _ un . simbolus

In [8]:
letters = [
    ['A',[0,1,0,1,0,1,1,1,1,1,0,1,1,0,1],[1,0,0,0]],
    ['E',[1,1,1,1,0,0,1,1,0,1,0,0,1,1,1],[1,0,0,0]],
    ['D',[1,1,0,1,0,1,1,0,1,1,0,1,1,1,0],[1,0,0,0]],
    ['B',[1,1,0,1,0,1,1,1,0,1,0,1,1,1,0],[0,1,0,0]],
    ['C',[1,1,1,1,0,0,1,0,0,1,0,0,1,1,1],[0,1,0,0]],
    ['F',[1,1,1,1,0,0,1,1,0,1,0,0,1,0,0],[0,1,0,0]],
    ['G',[1,1,1,1,0,0,1,0,1,1,0,1,1,1,1],[0,0,1,0]],
    ['H',[1,0,1,1,0,1,1,1,1,1,0,1,1,0,1],[0,0,1,0]],
    ['I',[0,1,0,0,1,0,0,1,0,0,1,0,0,1,0],[0,0,1,0]],
    
    ['!',[0,1,0,0,1,0,0,1,0,0,0,0,0,1,0],[0,0,0,1]],
    ['_',[0,0,0,0,0,0,0,0,0,0,0,0,1,1,1],[0,0,0,1]],
    ['.',[0,0,0,0,0,0,0,0,0,0,0,0,0,1,0],[0,0,0,1]],
]


class NeironHome:
    def __init__(self, in_size):
        self.weights = np.zeros(in_size)
        self.w0 = 0

    def classify(self, incoming_signals):
        net = np.dot(incoming_signals, self.weights) + self.w0
        
        return a_sigmoid(net)
    
    def learn(self, incoming_signals, c):
        y = self.classify(incoming_signals)
        delta = c - y
        
        if delta:
            for i in range(len(self.weights)):
                self.weights[i] += delta * learning_rate * incoming_signals[i]

            self.w0 += delta * learning_rate

In [9]:
run_neirons(letters, NeironHome, 15, 4, 1000)

A (0.9975535051097201, 6.660629571448497e-06, 0.008248493415553408, 0.00013232113048282796)
E (0.8293688905071558, 0.07107663617372244, 0.00345475745807826, 0.0008908342725369956)
D (0.8834289329308431, 0.040197189905192325, 0.005393751338105291, 0.000374360388021805)
B (0.06296538404255056, 0.9611077227614221, 2.5487150667717718e-05, 0.00044414160389372084)
C (0.08063124850588929, 0.941473518306242, 0.00588321544731504, 0.0016078235509806357)
F (0.02633476838322883, 0.9911138221082045, 0.0034084107149066696, 2.9723027415453927e-05)
G (0.04066958299013252, 0.00015968439826207983, 0.9849054420714136, 0.0002711874602893066)
H (0.02117992296299061, 1.3000067100018376e-05, 0.9923537292619337, 1.8435006117880236e-05)
I (0.0037253089873815924, 2.5566882336613036e-05, 0.9800763983367607, 0.02047026562186848)
! (0.018415486080804077, 4.3871158090628676e-05, 0.013755496597630701, 0.9781077632338467)
_ (0.00838609151522248, 0.00038552142987693667, 0.0020254630608827185, 0.9968259609187382)
. (0.

**Ir redzams, ka "!", "\_" un "." simboli tiek klasificēti kā 4. klase. Citu burtu klasifikācija nemainās.**

Pārtaisīsim TensorFlow kodu, lai viņš klasificētu burtus:

In [10]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(15, kernel_regularizer=tf.keras.regularizers.l2(0.1), activation='sigmoid', input_shape=(15, )))
model.add(tf.keras.layers.Dense(4, activation='softmax'))
model.compile(optimizer='adam', loss='binary_crossentropy')

np_letters = np.array(tuple(np.array(letter[1]) for letter in letters))
print(np_letters)
targets = np.array(tuple(np.array(letter[2]) for letter in letters))
print(targets)

model.fit(np_letters, targets, epochs=1000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[0 1 0 1 0 1 1 1 1 1 0 1 1 0 1]
 [1 1 1 1 0 0 1 1 0 1 0 0 1 1 1]
 [1 1 0 1 0 1 1 0 1 1 0 1 1 1 0]
 [1 1 0 1 0 1 1 1 0 1 0 1 1 1 0]
 [1 1 1 1 0 0 1 0 0 1 0 0 1 1 1]
 [1 1 1 1 0 0 1 1 0 1 0 0 1 0 0]
 [1 1 1 1 0 0 1 0 1 1 0 1 1 1 1]
 [1 0 1 1 0 1 1 1 1 1 0 1 1 0 1]
 [0 1 0 0 1 0 0 1 0 0 1 0 0 1 0]
 [0 1 0 0 1 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]]
[[1 0 0 0]
 [1 0 0 0]
 [1 0 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 1 0 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 1]]
[[0.27214292 0.27093387 0.2581872  0.198736  ]
 [0.26751614 0.2730208  0.25528914 0.20417398]
 [0.27163666 0.27308437 0.25628433 0.19899464]
 [0.27163467 0.27549928 0.2546564  0.19820963]
 [0.26433432 0.26913524 0.25392833 0.21260217]
 [0.2691185  0.2764809  0.25494376 0.19945689]
 [0.2716993  0.27473295 0.2595326  0.19403519]
 [0.2760262  0.27877855 0.26195794 0.18323733]
 [0.2159902  0.20971563 0.23432924 0.33996493]
 [0.21530646 0.20914814 0.2321186  0.3434268 ]


**Interesanti**, bet TensorFlow versija apstiprināja manas šaubas par "I" un "!" lidzīguma problēmu, kāmēr bez TensorFlow klasifikācija nostrādāja pareizi. Ar parējiem burtiem arī ir redzamas problēmas - B tiek pieskaitīts pie 2. klases. Pārbaudīšu, vai epohu skaits uzlabos situāciju:

In [11]:
model.fit(np_letters, targets, epochs=10000, verbose=0)
predictions = model.predict(np_letters)

print(predictions)

[[7.62309015e-01 5.89078106e-02 1.74614027e-01 4.16918285e-03]
 [2.94512719e-01 5.98162651e-01 9.62203741e-02 1.11041935e-02]
 [6.17455900e-01 2.51913726e-01 1.22427709e-01 8.20266269e-03]
 [2.59231746e-01 7.23216772e-01 1.32025220e-02 4.34897188e-03]
 [2.06596822e-01 7.22618341e-01 5.36366478e-02 1.71481632e-02]
 [8.64592344e-02 8.95993352e-01 1.24901561e-02 5.05730277e-03]
 [1.87915459e-01 2.38456614e-02 7.82886863e-01 5.35204541e-03]
 [1.20502479e-01 1.05892411e-02 8.63352656e-01 5.55568561e-03]
 [3.58347259e-02 8.43089074e-03 7.47584760e-01 2.08149597e-01]
 [1.34250745e-02 1.04186656e-02 7.49970749e-02 9.01159167e-01]
 [2.69288616e-03 2.80639832e-03 9.49602854e-03 9.85004663e-01]
 [7.07620173e-04 1.48930133e-03 4.18383395e-03 9.93619263e-01]]
